<a href="https://colab.research.google.com/github/whyzhuce/XConparraison/blob/master/Hull_%26_White_1F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implémentation du modèle Hull & White 1 facteur

> **Antonin Chaix**

L'objectif de ce colab est l'implémentation du modèle Hull & White 1 facteur, parfois dénommé également _HJM gaussien 1 facteur_, _Linear Gauss Markov_ (LGM 1F) ou encore _Extended Vasicek_.

Cette implémentation consiste en :

* La définition du modèle, le pricing des instruments standards de calibration dans le modèle (caplets/floorlets et swaptions) et la calibration du modèle sur le prix de marché de ces instruments.

* L'implémentation du pricing par Monte Carlo dans le modèle

* L'implémentation du pricing par différences finies dans le modèle

* L'utilisation des méthodes numériques précédentes pour évaluer, après une calibration adaptée du modèle, une série d'instruments vanilles et exotiques (retrouver par Monte Carlo ou différences finies le prix des caplets / floorlets / swaptions sur lesquels le modèle est calibré, évaluer une mid-curve option, une swaption bermuda etc...)

**NB** : ce colab ne présente pas le modèle HW1F en détails. Tout ce qu'il faut savoir sur le modèle (équations et paramètres, formule de reconstruction, formules fermées pour les instruments de calibration, procédure de calibration et méthodes numériques) figure dans la partie 6 et les annexes de ce [support de cours](https://drive.google.com/file/d/18-V9YqRfKlNW1UQ_wRCZR4RhGdQkuFed/view?usp=sharing).


## Fonctions de pricing dans les modèles Black & gaussien

Ci-dessous les fonctions standard de pricing dans les modèles de Black et gaussien (normal / Bachelier) utiles à l'implémentation du modèle Hull & White.

Noter à chaque fois les deux versions implémentées : une qui sera plus efficace pour le calcul d'un ou quelques prix, et une version numpy qui pourra être appelée sur des array numpy de paramètres et qui sera nettement plus efficace qu'une boucle dès que la taille des arrays en input augmentera...

In [ ]:
"""
Black & normal pricing formulas

"""
import numpy as np
import math
from scipy.stats import norm

# standard version : T, K, F, sigma must be single floats
def bs_call(T, K, F, sigma) :
	if (T==0 or sigma==0) : return max(F-K, 0)
	sigma_sqrt_T = sigma * math.sqrt(T)
	d1 = (math.log(F/K) + 0.5 * sigma**2 * T) / sigma_sqrt_T
	d2 = d1 - sigma_sqrt_T
	return F * norm.cdf(d1) - K * norm.cdf(d2)

# numpy compatible version (slower for non arrays, faster for large arrays)
def bs_call_np(T, K, F, sigma) :
	sigma_sqrt_T = sigma * np.sqrt(T)
	d1 = (np.log(F/K) + 0.5 * sigma**2 * T) / sigma_sqrt_T
	d2 = d1 - sigma_sqrt_T
	return F * norm.cdf(d1) - K * norm.cdf(d2)

# standard version : T, K, F, sigma must be single floats
def bs_put(T, K, F, sigma) :
	return bs_call(T, K, F, sigma) - F + K

# numpy compatible version (slower for non arrays, faster for large arrays)
def bs_put_np(T, K, F, sigma) :
	return bs_call_np(T, K, F, sigma) - F + K

# standard version : T, K, F, sigma must be single floats
def bs_option(call_put, T, K, F, sigma) :
	if (call_put == 'call') : return bs_call(T, K, F, sigma)
	else : return bs_put(T, K, F, sigma)

# numpy compatible version (slower for non arrays, faster for large arrays)
def bs_option_np(call_put, T, K, F, sigma) :
	return np.where (call_put == 'call', bs_call_np(T, K, F, sigma), bs_put_np(T, K, F, sigma))

# standard version : T, K, F, sigma must be single floats
def norm_call(T, K, F, sigma) :
	if (T==0 or sigma==0) : return max(F-K, 0)
	sigma_sqrt_T = sigma * math.sqrt(T)
	d = (F - K) / sigma_sqrt_T
	return sigma_sqrt_T * (d * norm.cdf(d) + norm.pdf(d))

# numpy compatible version (slower for non arrays, faster for large arrays)
def norm_call_np(T, K, F, sigma) :
	sigma_sqrt_T = sigma * np.sqrt(T)
	d = (F - K) / sigma_sqrt_T
	return sigma_sqrt_T * (d * norm.cdf(d) + norm.pdf(d))

# standard version : T, K, F, sigma must be single floats
def norm_put(T, K, F, sigma) :
	return norm_call(T, K, F, sigma) - F + K

# numpy compatible version (slower for non arrays, faster for large arrays)
def norm_put_np(T, K, F, sigma) :
	return norm_call_np(T, K, F, sigma) - F + K

# standard version : T, K, F, sigma must be single floats
def norm_option(call_put, T, K, F, sigma) :
	if (call_put == 'call') : return norm_call(T, K, F, sigma)
	else : return norm_put(T, K, F, sigma)

# numpy compatible version (slower for non arrays, faster for large arrays)
def norm_option_np(call_put, T, K, F, sigma) :
	return np.where (call_put == 'call', norm_call_np(T, K, F, sigma), norm_put_np(T, K, F, sigma))

## La courbe zéro-coupon

La courbe zéro-coupon, bootstrapée à partir des instruments de taux liquides (deposits, futures, swaps), est un input du modèle.

Ci-dessous une petite classe toute simple fournissant pour toute date $T$ le discount factor de maturité $T$ obtenu par interpolation des taux zéro-coupon fournis pour construire la courbe zéro-coupon.

In [ ]:
"""
Zero-coupon curve

"""
# on conserve à chaque fois tous les imports nécessaires (utile si ces différents morceaux de codes sont dans des fichiers .py distincts)
import numpy as np
from scipy.interpolate import interp1d

class zc_curve :

	def __init__(self, maturities, zc_rates):
		self.maturities = maturities
		self.zc_rates = zc_rates
		self.zc_rates_interp = interp1d(maturities, zc_rates, kind='cubic', fill_value="extrapolate")

	def df(self, T):
		return np.exp(-self.zc_rates_interp(T)*T)

## Les instruments de calibration

Les méthodes `price_from_phi`, `calibrate_phi` et `calibrate_sigma` de la classe `hw1f_model` implémentée ensuite travaillent sur des instruments de calibration (caplets/floorlets ou swaption) ou sur des sets d'instruments de calibration : il faut donc définir ces instruments. C'est l'objet de la classe suivante.

In [ ]:
"""
Vanilla instrument = caplet / floorlet or swaption

"""
import numpy as np

class instrument :

    def __init__(self, call_put, exp_time, start_time, pay_times, year_fractions, strike, nominal = 100.):
        self.call_put = call_put
        self.exp_time = exp_time
        self.start_time = start_time
        self.pay_times = pay_times
        self.year_fractions = year_fractions
        self.strike = strike
        self.nominal = nominal

    @classmethod
    def swaption(cls, payer_receiver, expiry, tenor, strike, nominal = 100.):
        call_put = 'call' if payer_receiver == 'payer' else 'put'
        start_time = expiry + 2./365.
        pay_times = np.arange(start_time+1, expiry+tenor+1)
        year_fractions = np.full(len(pay_times), 1.)
        return cls(call_put, expiry, start_time, pay_times, year_fractions, strike, nominal)

    @classmethod
    def capletfloorlet(cls, caplet_floorlet, expiry, tenor, strike, nominal = 100.):
        call_put = 'call' if caplet_floorlet == 'caplet' else 'put'
        start_time = expiry + 2./365.
        pay_times = np.array([start_time + tenor])
        year_fractions = np.array([tenor*365./360.])
        return cls(call_put, expiry, start_time, pay_times, year_fractions, strike, nominal)

    # store DFs values to save computation time
    def set_market_data(self, df, normal_vol):
        self.df_exp_time = df(self.exp_time)
        self.df_start_time = df(self.start_time)
        self.df_pay_times = df(self.pay_times)
        self.normal_vol = normal_vol

    def market_price(self):
        level = np.sum(self.df_pay_times * self.year_fractions)
        fwd = (self.df_start_time - self.df_pay_times[-1]) / level
        return self.nominal * level * norm_option(self.call_put, self.exp_time, self.strike, fwd, self.normal_vol)

    def forward(self):
        level = np.sum(self.df_pay_times * self.year_fractions)
        return (self.df_start_time - self.df_pay_times[-1]) / level

    def pv_underlying(self): # pv of underlying swap or FRA
        level = np.sum(self.df_pay_times * self.year_fractions)
        fwd = (self.df_start_time - self.df_pay_times[-1]) / level
        cp = 1. if (self.call_put=='call') else -1.
        return cp * self.nominal * level * (fwd - self.strike)

    def print(self):
        print("----------------------------------------------------------------------------------")
        print("VANILLA INSTRUMENT | Expiry = {} | Tenor = {:.2f} | strike = {:.4f}% | nominal = {}".format(self.exp_time, self.pay_times[-1] - self.start_time, self.strike*100., self.nominal))
        if hasattr(self, 'normal_vol'):
            print("Forward = {:.4f}% | Normal vol = {:.4f}% | Price = {:.4f}".format(self.forward()*100, self.normal_vol*100, self.market_price()))
        print()

## La classe Hull & White 1F

Il s'agit dans cette classe :

* De définir les paramètres du modèle : mean reversion $\lambda$ et volatilité instantanée $\sigma(t)$ du taux court, constante par morceaux

* D'implémenter les formules d'évaluation, dans le modèle, des instruments de calibration standard : caplets/floorlets et swaptions définis via la classe` instrument`

* D'implémenter la calibration de la fonction de volatilité du modèle $\sigma(t)$ constante par morceaux sur un set d'instruments de calibration consistant en une série de caplets/floorlets et/ou swaptions de maturités distinctes



In [ ]:
"""
Description & calibration of Hull & White 1 factor model

"""
import numpy as np
import math
from scipy import optimize
from scipy.interpolate import interp1d
# from pricing_formulas import norm_option, bs_option, bs_option_np # requis si code dans un fichier .py standalone

# Hull & White 1 factor
class hw1f_model :

	def __init__(self, lam = None, t = None, sigma_t = None):
		self.lam = lam
		self.t = t
		self.sigma_t = sigma_t

	def print(self):
		print("------------------------------------")
		print("HW1F MODEL | Mean reversion = {:.2f}%\nVol sigma(t) (%) = ".format(self.lam*100.))
		with np.printoptions(precision=4, suppress=True):
			print(np.column_stack((self.t, self.sigma_t*100)))
		print()


	# computes phi(t) = var(Xt) on time vector t (np.array[(n,)])
	def get_phi(self, t):
		new_t = np.append(self.t, t)
		new_t = np.delete(new_t, np.where(new_t>t[-1]))
		new_t = np.unique(np.sort(new_t))
		sigma = interp1d(self.t, self.sigma_t, kind='previous', fill_value="extrapolate")
		sigma_t = sigma(new_t)
		phi_new_t = np.empty(len(new_t))
		phi_new_t[0] = 0.
		exp_2_lam_t = np.exp(2.*self.lam*new_t)

		for i in range(1, len(new_t)):
			phi_new_t[i] = (phi_new_t[i-1] * exp_2_lam_t[i-1] + sigma_t[i-1]**2 * (exp_2_lam_t[i] - exp_2_lam_t[i-1])/(2.*self.lam)) / exp_2_lam_t[i]

		phi = interp1d(new_t, phi_new_t, kind='previous', fill_value="extrapolate")

		return phi(t)


	# HW1F price for calib instrument instr and for variance phi
	def price_from_phi (self, instr, phi):
		Tf  = instr.exp_time
		T0  = instr.start_time
		DF0 = instr.df_start_time
		DFs = instr.df_pay_times

		# caplet
		if (len(instr.pay_times) == 1) :
			T1  = instr.pay_times[0]
			delta = instr.year_fractions[0]
			vol = math.sqrt(phi / Tf) * (math.exp(-self.lam*(T0-Tf)) - math.exp(-self.lam*(T1-Tf))) / self.lam
			return instr.nominal * DFs[0] * bs_option(instr.call_put, Tf, 1. + delta * instr.strike, DF0 / DFs[0], vol)

		# swaption = bond option
		else :
			# bond option coefficients
			coefs = instr.year_fractions * instr.strike
			coefs[-1] += 1

			# precalcs
			beta0 = (1.- math.exp(-self.lam * (T0-Tf)) ) / self.lam
			betas = (1.- np.exp(-self.lam * (instr.pay_times-Tf)) ) / self.lam
			DFs_sur_DF0 = DFs / DF0
			betas_moins_beta0 = betas - beta0
			betas2_moins_beta02 = betas**2 - beta0**2

			# func to solve to find exercise frontier
			def f(x):
				return np.sum(coefs * DFs_sur_DF0 * np.exp(-0.5 * betas2_moins_beta02 * phi - betas_moins_beta0 * x)) - 1

			def fprime(x):
				return np.sum(- coefs * DFs_sur_DF0 * betas_moins_beta0 * np.exp(-0.5 * betas2_moins_beta02 * phi - betas_moins_beta0 * x))

			# DL de l'exponentielle pour trouver un bon guess
			guess_x = (-1 + np.sum(coefs * DFs_sur_DF0 * (1 - 0.5 * betas2_moins_beta02 * phi))) / np.sum(coefs * DFs_sur_DF0 * betas_moins_beta0)

			x0 = optimize.newton(f, guess_x, fprime)

			strikes = DFs_sur_DF0 * np.exp(-0.5 * betas2_moins_beta02 * phi - betas_moins_beta0 * x0)
			vols = math.sqrt(phi / Tf) * (math.exp(-self.lam*(T0-Tf)) - np.exp(-self.lam*(instr.pay_times-Tf))) / self.lam
			call_put = 'put' if (instr.call_put == 'call') else 'call'
			return instr.nominal * DF0 * np.sum(coefs * bs_option_np(call_put, Tf, strikes, DFs_sur_DF0, vols))

	# find variance phi for a given calibration instrument
	def calibrate_phi(self, instr):

		market_price = instr.market_price()

		def f(phi):
			return self.price_from_phi(instr, phi) - market_price

		def fprime(phi):
			eps = 1e-06
			return (f(phi+eps) - f(phi-eps)) / (2.*eps)

		guess_phi = instr.normal_vol**2 * instr.exp_time # guess déterminé en supposant que vol r_t = vol taux swap
		return optimize.newton(f, guess_phi, fprime)


	# calibrate sigma(t) term structure on a set of calibration instruments
	def calibrate_sigma (self, calib_set):

		phi = []
		for instr in calib_set:
			phi.append(self.calibrate_phi(instr))

		t = np.empty(len(calib_set)+1)
		sigma_t = np.empty(len(calib_set)+1)

		t[0] = 0;
		last_phi = 0.;
		sigma_min = 0.0005

		for i in range(len(calib_set)):

			t[i+1] = calib_set[i].exp_time
			exp = math.exp(-2.*self.lam*(t[i+1]-t[i]))
			first_term = exp * last_phi

			# var squeeze
			if (phi[i] < first_term):
				sigma_t[i] = sigma_min
				phi[i] = first_term + sigma_min**2 * (1.-exp) / (2.*self.lam) # update phi accordingly
				print("HW1F calibration : variance squeeze at step {}. Instrument [expiry = {} / tenor = {}] will be mispriced".format(i+1, calib_set[i].exp_time, calib_set[i].pay_times[-1] - calib_set[i].start_time))
			# everyting ok
			else :
				sigma_t[i] = math.sqrt( (phi[i]-first_term) * 2.*self.lam / (1.-exp) )
				if (sigma_t[i]<sigma_min) :
					sigma_t[i] = sigma_min
					phi[i] = first_term + sigma_min * sigma_min * (1.0-exp) / (2.*self.lam)
					print("HW1F calibration : variance squeeze at step {}. Instrument [expiry = {} / tenor = {}] will be mispriced".format(i+1, calib_set[i].exp_time, calib_set[i].pay_times[-1] - calib_set[i].start_time))

			last_phi = phi[i]

		# extropolate after last exp date => same size for t & sigma_t
		sigma_t[-1] = sigma_t[-2]

		# store in class attributes
		self.t = t
		self.sigma_t = sigma_t

	# price of vanilla instrument (slow, just for test & display purpose, not used in calibration)
	def price(self, instr):
		t = np.array([instr.exp_time])
		phi = float(self.get_phi(t))
		return self.price_from_phi(instr, phi)


## Monte Carlo dans le modèle HW1F

La classe `hw1f_mc` ci-dessous implémente le pricing par Monte Carlo dans le modèle Hull & White 1 facteur.

L'instanciation de la classe crée la discrétisation en temps et génère les trajectoires de la variable d'état $X_t=r_t - f(0,t)$.

Le pricing lui-même s'effectue ensuite via la méthode `price`.

In [ ]:
"""
Hull & White 1 factor Monte Carlo

"""
import numpy as np
import math
from scipy.interpolate import interp1d

class hw1f_mc :

	def __init__(self, df, hw, Nsimul, dt, event_t):

		self.df_yc = df
		self.hw = hw
		self.Nsimul = Nsimul

		# MC time steps
		t_max = event_t[-1]
		mc_t = np.arange(0, t_max + dt, dt)

		# merge mc + event + hw times
		t = np.concatenate((mc_t, event_t, hw.t))
		t = np.delete(t, np.where(t>t_max)) # remove times larger that monte carlo t_max
		t = np.unique(np.sort(t))
		self.t = t # t as attribute for get_index method

		# some display...
		print("HW1F Monte Carlo\n=====================================")
		print("{:,} simulations | {} time steps".format(Nsimul, len(t)-1).replace(',', ' '))
		print("schedule = {}\n".format(t))

		# compute HW sigma(t) on MC schedule
		sigma = interp1d(hw.t, hw.sigma_t, kind='previous', fill_value="extrapolate")
		sigma_t = sigma(t)

		# compute phi(t) on MC schedule
		phi_t = hw.get_phi(t)
		self.phi_t = phi_t # phi_t as attribute for reconstruction formula

		# compute DFs on MC schedule
		df_t = df(t)

		# MC Nsteps
		Nsteps = len(t) - 1

		# generate random N(0,1)
		normal = np.random.normal(0, 1, (Nsteps, Nsimul))

		# state variable X array (MC time schedule x simulations)
		self.X = np.empty(shape=(Nsteps+1, Nsimul)) # X as attribute for reconstr. formula
		self.X[0,:] = 0.

		# terme d'actualisation exp(-int r_t dt)
		self.disc = np.empty(shape=(Nsteps+1, Nsimul)) # disc as attribute for discounted payoff calculations outside class
		self.disc[0,:] = 1

		# precalc exp(-lambda * ti)
		exp_lam_t = np.exp(hw.lam*t)

		# precalc of stdev factor of N(0,1) in X(t)
		stdev = sigma_t[:-1] * np.sqrt( (exp_lam_t[1:]**2 - exp_lam_t[:-1]**2) / (2.*hw.lam))

		# MC main loop : generate X and discount
		for i in range(Nsteps):
			int_exp_phi = 0.5 * (exp_lam_t[i]*phi_t[i] + exp_lam_t[i+1]*phi_t[i+1]) * (t[i+1] - t[i])
			self.X[i+1,:] = ( self.X[i,:] * exp_lam_t[i] + int_exp_phi + stdev[i] * normal[i,:] ) /  exp_lam_t[i+1]
			self.disc[i+1,:] = self.disc[i,:] * (df_t[i+1]/df_t[i]) * np.exp(-0.5 * (self.X[i+1,:] + self.X[i,:]) * (t[i+1] - t[i]))


	def get_index(self, time):
		if (time in self.t) : return int(np.where(np.isclose(self.t,time))[0])
		else: exit("HW1F Monte Carlo : get_index : time {} does not exist in MC schedule\n".format(time))

	# reconstruction formula
	def df(self, time, maturity):
		i = self.get_index(time)
		beta = (1. - math.exp(-self.hw.lam*(maturity-time))) / self.hw.lam
		return (self.df_yc(maturity)/self.df_yc(time)) * np.exp(-0.5 * beta**2 * self.phi_t[i] - beta * self.X[i,:])

	# discount term
	def discount(self, time):
		i = self.get_index(time)
		return self.disc[i,:]

	# shortcut for payoff of swaption or caplet/floorlet
	def discounted_payoff(self, instr):
		df_start_time = self.df(instr.exp_time, instr.start_time)
		df_pay_times = np.empty(shape=(len(instr.pay_times), self.Nsimul))
		for i in range(len(instr.pay_times)):
			df_pay_times[i,:] = self.df(instr.exp_time, instr.pay_times[i])

		level = np.sum(df_pay_times * instr.year_fractions[:, np.newaxis], axis=0)
		swaprate = (df_start_time - df_pay_times[-1,:]) / level
		cp = 1. if instr.call_put == 'call' else -1.
		return self.discount(instr.exp_time) * instr.nominal * level * np.maximum(cp * (swaprate - instr.strike), 0.)


	# compute price, confidence interval & display
	def compute_price(self, discounted_payoff, option_name, closed_form_price = -1, display=True) :
		price = np.mean(discounted_payoff)
		if (display):
			Nsimul = len(discounted_payoff)
			stdev = np.std(discounted_payoff)
			IC_low = price - 1.96 * stdev / math.sqrt(Nsimul)
			IC_up = price + 1.96 * stdev / math.sqrt(Nsimul)
			in_out = ''
			print("------------------------------------------------------")
			if (closed_form_price == -1) :
				print("{} = {:.5f}".format(option_name, price) )
			else :
				print("{} = {:.5f} (closed-form price = {:.5f})".format(option_name, price, closed_form_price) )
				in_out = ' - IN' if (IC_low < closed_form_price < IC_up) else ' - OUT'
			print("Conf. interv. 95% = [{:.5f} ; {:.5f}]{}".format(IC_low, IC_up, in_out) )
			if (closed_form_price != -1):
				print("Rel. error : {:.4f}%".format(100.*(price- closed_form_price)/closed_form_price))
			print()

		return price

## Calibration du modèle et repricing des instruments par Monte Carlo

Le code suivant :

* Réalise la calibration du modèle HW1F sur une diagonale de swaptions 2y/13y, 3y/12y, ..., 5y/10y, ..., 14y/1y de strike 0.25% (market vol gaussienne constante = 0.65%)

* Reprice chacune des swaptions en Monte Carlo dans le modèle et affiche :
  * Le prix Monte Carlo de la swaption (1 000 000 simulations)
  * Le prix Hull & White de la swaption, obtenu par formule fermée (utilisé lors de la phase de calibration)
  * Le prix de marché de la swaption

Modulo la petite erreur statistique du Monte Carlo, on constate que les 3 prix coïncident pour l'ensemble des swaptions du set de calibration.

In [ ]:
"""
Calibration à une diagonale de swaptions
Et repricing des swaptions par Monte Carlo

"""
import numpy as np
import math
import time
# requis si code dans un fichier .py standalone :
# from hw1f import hw1f_model, instrument
# from hw1f_mc import hw1f_mc
# from zc_curve import zc_curve

t0  = time.time()

# yield curve t = 0
maturities = np.array([0, 1, 2, 5, 10, 15, 20, 30])
zc_rates = np.array([-0.55, -0.52, -0.51, -0.4,  -0.18,  0.02,  0.11,  0.09]) * 0.01
yc = zc_curve(maturities, zc_rates)

# calibration to diagonal swaption set 2y/13y => 14y/1y
total_mat = 15
calib_set = []
for expiry in range (2, total_mat) :
	tenor = total_mat - expiry
	swaption = instrument.swaption('payer', expiry, tenor, 0.0025, 100.)
	swaption.set_market_data(yc.df, 0.0065)
	calib_set.append(swaption)

# create hw1f model & calibrate
mean_reversion = 0.015
hw = hw1f_model(mean_reversion)
hw.calibrate_sigma(calib_set)
hw.print()

t1  = time.time()

# product event times => based on calib set as we reprice the calib set swaptions
event_t = []
for swaption in calib_set :
	event_t.append(swaption.exp_time)

# MC params
Nsimul = 10**6
dt = 1.

# generate MC simulations
mc = hw1f_mc(yc.df, hw, Nsimul, dt, event_t)

# Reprice all swaptions in calibration set
for swaption in calib_set: # in calib_set[3:4] to price only 5y/10y
	mc.compute_price(mc.discounted_payoff(swaption), 'Swaption {}y/{}y'.format(swaption.exp_time, int(swaption.pay_times[-1]-swaption.start_time)), hw.price(swaption))
	print("Swaption market price = {:.4f}\n".format(swaption.market_price()))

# mid-curve option 5/5/5 => depends on mean reversion value
mid_curve_option = instrument('call', 5, 10, np.arange(11, 16), np.full(5, 1.), 0.005, 100)
mc.compute_price(mc.discounted_payoff(mid_curve_option), 'Mid curve option (mean rev = {}%)'.format(hw.lam*100))

t2 = time.time()

print("\nCalibration time = {:.2f} seconds".format(t1 - t0))
print("Monte Carlo time = {:.2f} seconds".format(t2- t1))
print("TOTAL = {:.2f} seconds\n".format(t2- t0))

------------------------------------
HW1F MODEL | Mean reversion = 1.50%
Vol sigma(t) (%) = 
[[ 0.      0.7333]
 [ 2.      0.7294]
 [ 3.      0.7271]
 [ 4.      0.7253]
 [ 5.      0.7234]
 [ 6.      0.7215]
 [ 7.      0.7198]
 [ 8.      0.7184]
 [ 9.      0.7174]
 [10.      0.7171]
 [11.      0.7172]
 [12.      0.7175]
 [13.      0.7178]
 [14.      0.7178]]

HW1F Monte Carlo
1 000 000 simulations | 14 time steps
schedule = [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]

------------------------------------------------------
Swaption 2y/13y = 3.90523 (closed-form price = 3.91644)
Conf. interv. 95% = [3.89367 ; 3.91680] - IN
Rel. error : -0.2860%

Swaption market price = 3.9164

------------------------------------------------------
Swaption 3y/12y = 4.84241 (closed-form price = 4.85507)
Conf. interv. 95% = [4.82900 ; 4.85582] - IN
Rel. error : -0.2607%

Swaption market price = 4.8551

------------------------------------------------------
Swaption 4y/11y = 5.43661 (closed

Ci-dessous on reproduit exactement la même chose avec une calibration sur une série de caplets EURIBOR 12M de strike 0.15% (market vol constante 0.75%), de maturité 1y jusque 19y.

Autrement dit, on calibre le modèle, caplet par caplet; sur un CAP EURIBOR 12M de maturité 20 ans et de strike 0.15%, dont la vol (normale) de marché est supposée égale à 0.75%.

In [ ]:
"""
Calibration du HW1F à un strip de caplets (Cap 20 ans Euribor 12m)
Et repricing des caplets par Monte Carlo

"""
import numpy as np
import math
import time
# requis si code dans un fichier .py standalone :
# from hw1f import hw1f_model, instrument
# from hw1f_mc import hw1f_mc
# from zc_curve import zc_curve

t0  = time.time()

# yield curve t = 0
maturities = np.array([0, 1, 2, 5, 10, 15, 20, 30])
zc_rates = np.array([-0.55, -0.52, -0.51, -0.4,  -0.18,  0.02,  0.11,  0.09]) * 0.01
yc = zc_curve(maturities, zc_rates)

# caplets
total_mat = 20
calib_set = []
for expiry in range (1, total_mat) :
	caplet = instrument.capletfloorlet('caplet', expiry, 1., 0.0015, 100.)
	caplet.set_market_data(yc.df, 0.0075)
	calib_set.append(caplet)

# create hw1f model & calibrate
mean_reversion = 0.015
hw = hw1f_model(mean_reversion)
hw.calibrate_sigma(calib_set)
hw.print()

t1  = time.time()

# product event times => based on calib set as we reprice the calib set swaptions
event_t = []
for caplet in calib_set :
	event_t.append(caplet.exp_time)

# MC params
Nsimul = 10**6
dt = 1.

# generate MC simulations
mc = hw1f_mc(yc.df, hw, Nsimul, dt, event_t)

# Reprice all caplets in calibration set
for caplet in calib_set: # in calib_set[3:4] to price only 5y/10y
	mc.compute_price(mc.discounted_payoff(caplet), 'Caplet {}y'.format(caplet.exp_time), hw.price(caplet))
	print("Caplet market price = {:.4f}\n".format(caplet.market_price()))

t2 = time.time()

print("\nCalibration time = {:.2f} seconds".format(t1 - t0))
print("Monte Carlo time = {:.2f} seconds".format(t2- t1))
print("TOTAL = {:.2f} seconds\n".format(t2- t0))

------------------------------------
HW1F MODEL | Mean reversion = 1.50%
Vol sigma(t) (%) = 
[[ 0.      0.7733]
 [ 1.      0.7842]
 [ 2.      0.7943]
 [ 3.      0.8045]
 [ 4.      0.8149]
 [ 5.      0.8252]
 [ 6.      0.8351]
 [ 7.      0.8449]
 [ 8.      0.8545]
 [ 9.      0.8639]
 [10.      0.8737]
 [11.      0.8838]
 [12.      0.8942]
 [13.      0.9051]
 [14.      0.9161]
 [15.      0.9265]
 [16.      0.9366]
 [17.      0.9465]
 [18.      0.9564]
 [19.      0.9564]]

HW1F Monte Carlo
1 000 000 simulations | 19 time steps
schedule = [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19.]

------------------------------------------------------
Caplet 1y = 0.08347 (closed-form price = 0.08362)
Conf. interv. 95% = [0.08302 ; 0.08392] - IN
Rel. error : -0.1844%

Caplet market price = 0.0836

------------------------------------------------------
Caplet 2y = 0.20064 (closed-form price = 0.20085)
Conf. interv. 95% = [0.19980 ; 0.20147] - IN
Rel. error : -0.1068%


## Différences finies dans le modèle HW1F

La classe `hw1f_pde` ci-dessous implémente le pricing par différences finies  dans le modèle Hull & White 1 facteur (schéma de Crank-Nicholson permettant de résoudre numériquement l'EDP dont est solution le prix d'une option).

A noter que cet algorithme est très similaire à celui implementé dans [ce colab](https://colab.research.google.com/drive/190FfBc0w-fz8vim3x7RrFcb_Tu-Px2Hu?usp=sharing) (différences finies dans le cadre du modèle à volatilité locale).

In [ ]:
"""
Hull & White 1 factor finite differences

"""
import numpy as np
import math
import time
from scipy.interpolate import interp1d
from scipy.linalg import solve_banded

class hw1f_pde :

	def __init__(self, df, hw, nx, nstdev, dt, event_t):

		self.df_yc = df
		self.hw = hw

		# PDE times
		t_max = event_t[-1] # must be = last maturity of priced product
		dt_min = t_max / 50. # 50 time steps minimum !
		dt = min(dt, dt_min)
		pde_t = np.arange(0, t_max + dt, dt)

		# merge pde + prod + hw times
		t = np.concatenate((pde_t, event_t, hw.t))
		t = np.delete(t, np.where(t>t_max)) # remove times larger than PDE t_max
		t = np.unique(np.sort(t))
		self.t = t # t as attribute for get_index method

		# compute phi(t) on PDE schedule (as attribute : needed for reconstruction formula)
		self.phi_t = self.hw.get_phi(self.t)

		# x space
		x_max = nstdev * math.sqrt(self.phi_t[-1])
		x_min = - x_max
		self.x = np.linspace(x_min, x_max, nx, endpoint=True)



	# compute & display PDE price
	def compute_price (self, payoff, name = 'PDE price', closed_form_price = -1, display = True) :
		t_start = time.time()

		# Crank Nicholson
		theta = 0.5

		# compute HW sigma(t) on PDE schedule
		sigma = interp1d(self.hw.t, self.hw.sigma_t, kind='previous', fill_value="extrapolate")
		sigma_t = sigma(self.t)

		# compute DFs on PDE schedule
		df_t = self.df_yc(self.t)

		# space & time dimensions
		nt = len(self.t)
		nx = len(self.x)
		dx = (self.x[-1] - self.x[0]) / (nx - 1.)

		# option value
		v = np.zeros((nt, nx)) # final price will be v[0, int((nx -1)/2)]

		# keep memory of times where payoff is updated (in fact, where self.df is called...)
		# just for display
		self.event_times = []

		# payoff at final date
		v[-1,:] = payoff(self.t[-1], v[-1,:])

		# backward loop from t = t_max - dt to t = 0
		for i in range(nt-2, -1, -1): # => i = nt-2 ... 0

			# precalcs
			dt = self.t[i+1] - self.t[i]
			var =  sigma_t[i]**2 * dt
			mu = (0.5*(self.phi_t[i+1]+self.phi_t[i]) - self.hw.lam*self.x) * dt
			r = self.x * dt + math.log(df_t[i]/df_t[i+1])

			alpha_u = (var + mu * dx) / (2.*dx**2)
			alpha_d = (var - mu * dx) / (2.*dx**2)
			alpha_c = np.full(nx, - var / dx**2) # alpha_c does not depend upon x => cst vector

			# limits
			alpha_u[0]  = mu[0] / dx
			alpha_c[0]  = -alpha_u[0]
			alpha_d[0]  = 0.
			alpha_u[-1] = 0.
			alpha_c[-1] = mu[-1] / dx
			alpha_d[-1] = -alpha_c[-1]

			# transition coefs
			p_ur = (1.-theta)*alpha_u
			p_dr = (1.-theta)*alpha_d
			p_cr = (1.-theta)*alpha_c + 1. - (1.-theta)*r
			p_ul = -theta*alpha_u
			p_dl = -theta*alpha_d
			p_cl = -theta*alpha_c + 1. + theta*r

			# compute right member
			right = np.empty(nx)
			right[0] = p_cr[0]*v[i+1, 0] + p_ur[0]*v[i+1, 1]
			right[1:nx-1] = p_dr[1:nx-1]*v[i+1,0:nx-2] + p_cr[1:nx-1]*v[i+1,1:nx-1] + p_ur[1:nx-1]*v[i+1,2:nx]
			right[-1] = p_cr[-1]*v[i+1,-1] + p_dr[-1]*v[i+1,-2]

			# backward step inverting tridiagonal matrix
			upper_band = np.insert(p_ul, 0, 0.)[:-1] # transform p_ul to call solve_banded (0 must be in first position)
			lower_band = np.append(p_dl[1:nx], 0)  # transform p_ul to call solve_banded (0 must be in last position)
			tridiag = np.array([upper_band, p_cl, lower_band])
			v[i,:] = solve_banded((1, 1), tridiag, right)

			# apply payoff to v
			v[i,:] = payoff(self.t[i], v[i,:])

		price = v[0, int((nx-1)/2)]

		if (display):
			print("------------------------------------------------------")
			if (closed_form_price == -1) :
				print("{} = {:.5f}".format(name, price) )
			else :
				print("{} = {:.5f} (closed-form price = {:.5f})".format(name, price, closed_form_price) )
			if (closed_form_price != -1):
				print("Rel. error : {:.4f}%".format(100.*(price-closed_form_price)/closed_form_price))
			if (self.event_times):
				print("PDE DF function called in payoff at event time(s) : {}".format(set(self.event_times)))
			else :
				print("Caution : no call to df function in payoff !")
			print("PDE computation time : {:.3f} seconds\n".format(time.time()-t_start))

		return price


	# find index of time in PDE times (time must belong to PDE schedule !!)
	def get_index(self, time):
		if (time in self.t):
			return int(np.where(np.isclose(self.t,time))[0])
		else: exit("\nHW1F PDE get_index : time {} does not exist in PDE schedule !\n".format(time))

	# PDE df : returns DF values accross x (shape=(nx,))
	def df(self, t, T):
		i = self.get_index(t)
		self.event_times.append(t)
		beta = (1.-math.exp(-self.hw.lam*(T-t))) / self.hw.lam
		return (self.df_yc(T)/self.df_yc(t)) * np.exp(-0.5*beta**2*self.phi_t[i] - beta*self.x)

	# PDE df : version for a T vector => returns a (nx, len(T)) array
	def df_array(self, t, T):
		i = self.get_index(t)
		self.event_times.append(t)
		beta = (1.-np.exp(-self.hw.lam*(np.transpose(T)-t))) / self.hw.lam
		return (self.df_yc(T)/self.df_yc(t)) * np.exp(-0.5*beta**2*self.phi_t[i] - beta[np.newaxis,:]*self.x[:,np.newaxis])


	# pv of underlying swap / fra of a swaption / caplet
	def pv_underlying(self, instr, t):
		df_start_time = self.df(t, instr.start_time)
		df_pay_times = self.df_array(t, instr.pay_times)
		level = np.sum(df_pay_times * np.transpose(instr.year_fractions)[np.newaxis,:], axis=1)
		swaprate = (df_start_time - df_pay_times[:,-1]) / level
		cp = 1. if (instr.call_put=='call') else -1.
		return cp * instr.nominal * level * (swaprate - instr.strike)

## Calibration du modèle à un cap EURIBOR 6M 30 ans  et repricing des caplets par différences finies

Comme effectué un peu plus haut en Monte Carlo, l'objectif du code suivant est de calibrer le modèle à un cap Euribor 6M de maturité 30 ans (strike 1%, implied cap vol 0.65%), caplet par caplet, puis de retrouver le prix de chacun des caplet en l'évaluant via le schéma de différences finies implémenté précédemment.

In [ ]:
"""
Calibration à un cap 30y (caplet par caplet)
Et repricing en PDE des caplets

"""
import numpy as np
import math
import time
# requis si code dans un fichier .py standalone :
# from hw1f import hw1f_model, instrument
# from hw1f_pde import hw1f_pde
# from zc_curve import zc_curve

t0 = time.time()

# yield curve t = 0
maturities = np.array([0, 1, 2, 5, 10, 15, 20, 30])
zc_rates = np.array([-0.55, -0.52, -0.51, -0.4,  -0.18,  0.02,  0.11,  0.09]) * 0.01
yc = zc_curve(maturities, zc_rates)

# calibration sur set caplet euribor 6m
total_mat = 30
strike = 0.001
tenor = 0.5 # Euribor 6M
calib_set = []
for expiry in np.arange(0.5, total_mat, tenor) :
	caplet = instrument.capletfloorlet('caplet', expiry, tenor, strike, 100.)
	caplet.set_market_data(yc.df, 0.0065)
	calib_set.append(caplet)

#create hw1f model & calibrate
mean_rev = 0.02
hw = hw1f_model(mean_rev)
hw.calibrate_sigma(calib_set)
hw.print()
print("Calibration time = {:.3f} seconds\n\n".format(time.time() - t0))

# PDE params
nx = 601
nstdev = 5
dt = 0.05

# PDE price all caplets
for caplet in calib_set:
	def caplet_payoff(t, v_t):
		if (math.isclose(t, caplet.exp_time)):
			v_t = np.maximum(pde.pv_underlying(caplet, t), 0.)
		return v_t

	caplet.print()
	hw_price = hw.price(caplet)

	pde = hw1f_pde(yc.df, hw, nx, nstdev, dt, event_t = [caplet.exp_time])
	price = pde.compute_price(caplet_payoff, 'Caplet', hw_price)
	print("Market price : {:.5f}\n".format(caplet.market_price()))

------------------------------------
HW1F MODEL | Mean reversion = 2.00%
Vol sigma(t) (%) = 
[[ 0.      0.6664]
 [ 0.5     0.6729]
 [ 1.      0.6794]
 [ 1.5     0.6857]
 [ 2.      0.6918]
 [ 2.5     0.6979]
 [ 3.      0.704 ]
 [ 3.5     0.71  ]
 [ 4.      0.716 ]
 [ 4.5     0.7221]
 [ 5.      0.728 ]
 [ 5.5     0.7339]
 [ 6.      0.7397]
 [ 6.5     0.7455]
 [ 7.      0.7512]
 [ 7.5     0.7569]
 [ 8.      0.7625]
 [ 8.5     0.768 ]
 [ 9.      0.7735]
 [ 9.5     0.779 ]
 [10.      0.7845]
 [10.5     0.79  ]
 [11.      0.7956]
 [11.5     0.8011]
 [12.      0.8066]
 [12.5     0.8122]
 [13.      0.8178]
 [13.5     0.8233]
 [14.      0.829 ]
 [14.5     0.8345]
 [15.      0.84  ]
 [15.5     0.8453]
 [16.      0.8506]
 [16.5     0.8559]
 [17.      0.8611]
 [17.5     0.8663]
 [18.      0.8715]
 [18.5     0.8766]
 [19.      0.8817]
 [19.5     0.8868]
 [20.      0.8918]
 [20.5     0.8968]
 [21.      0.9018]
 [21.5     0.9067]
 [22.      0.9115]
 [22.5     0.9163]
 [23.      0.9211]
 [23.5     0.9

## Calibration du modèle à une diagonale de swaptions européenne, repricing par différences finies des swaptions européenne et de la swaption bermuda associée

On reprend la diagonale de swaptions européennes 2y/13y, 3y/12y, ..., 5y/10y, ..., 14y/1y utilisée précédemment, on calibre le modèle HW1F sur ces swaptions et on reprice par différences finies ces swaptions.

L'intérêt de la méthode des différences finies est que, contrairement au Monte Carlo, elle permet également de pricer la swaption bermuda associée (*Bermuda swaption 15y no call 2y*). C'est ce qui est fait à la fin de la cellule. On peut obtient au passage la valeur de la switch option = bermuda price - most expensive swaption.

In [ ]:
"""
Calibration à une diagonale de swaptions
Et repricing en PDE des swaptions et de la Bermuda correspondante

"""
import numpy as np
import math
import time
# requis si code dans un fichier .py standalone :
# from hw1f import hw1f_model, instrument
# from hw1f_pde import hw1f_pde
# from zc_curve import zc_curve

# yield curve t = 0
maturities = np.array([0, 1, 2, 5, 10, 15, 20, 30])
zc_rates = np.array([-0.55, -0.52, -0.51, -0.4,  -0.18,  0.02,  0.11,  0.09]) * 0.01
yc = zc_curve(maturities, zc_rates)

# calibration to diagonal swaption set 2y/13y => 14y/1y
total_mat = 15
calib_set = []
for expiry in range (2, total_mat) :
	tenor = total_mat - expiry
	swaption = instrument.swaption('payer', expiry, tenor, 0.0025, 100.)
	swaption.set_market_data(yc.df, 0.0065)
	calib_set.append(swaption)

# create hw1f model & calibrate
mean_reversion = 0.02
hw = hw1f_model(mean_reversion)
hw.calibrate_sigma(calib_set)
hw.print()

# PDE params
nx = 601
nstdev = 5
dt = 0.05

# Reprice all swaptions in calibration set
for swaption in calib_set: # in calib_set[3:4] to price only 5y/10y

	pde = hw1f_pde(yc.df, hw, nx, nstdev, dt, event_t = [swaption.exp_time])

	def swaption_payoff(t, v_t):
		if (math.isclose(t, swaption.exp_time)):
			v_t = np.maximum(pde.pv_underlying(swaption, t), 0.)
		return v_t

	pde.compute_price(swaption_payoff, 'Swaption {}y/{}y'.format(swaption.exp_time, int(swaption.pay_times[-1]-swaption.start_time)), hw.price(swaption))
	print("Swaption market price = {:.5f}\n".format(swaption.market_price()))


# Price bermuda swaption based upon calibration set
def bermuda_payoff(t, v_t):
	for swaption in calib_set:
		if (math.isclose(t, swaption.exp_time)):
			v_t = np.maximum(pde.pv_underlying(swaption, t), v_t)
			break
	return v_t

print("\nBERMUDA SWAPTION BASED UPON CALIBRATION SET")
# get event times = swaptions expiry times
event_t = []
for swaption in calib_set :
	event_t.append(swaption.exp_time)

pde = hw1f_pde(yc.df, hw, nx, nstdev, dt, event_t)
bermuda_price = pde.compute_price(bermuda_payoff, 'Bermuda swaption 15y no call 2y')

# find most expensive swaption & compute switch option value...
swaptions_prices = np.empty(len(calib_set))
for i in range(len(calib_set)):
	swaptions_prices[i] = calib_set[i].market_price()
imax = np.where(swaptions_prices == np.amax(swaptions_prices))[0][0]
me = calib_set[imax] # most expensive swaption
print('Most expensive swaption : {}y/{}y, price = {:.6f}'.format(me.exp_time, int(me.pay_times[-1]-me.start_time), me.market_price()) )
print('\nSwitch option = {:.6f}\n\n'.format(bermuda_price - me.market_price()))



------------------------------------
HW1F MODEL | Mean reversion = 2.00%
Vol sigma(t) (%) = 
[[ 0.      0.7602]
 [ 2.      0.7565]
 [ 3.      0.7542]
 [ 4.      0.7523]
 [ 5.      0.7503]
 [ 6.      0.7482]
 [ 7.      0.7462]
 [ 8.      0.7444]
 [ 9.      0.7429]
 [10.      0.7421]
 [11.      0.7416]
 [12.      0.7411]
 [13.      0.7406]
 [14.      0.7406]]

------------------------------------------------------
Swaption 2y/13y = 3.91648 (closed-form price = 3.91644)
Rel. error : 0.0012%
PDE DF function called in payoff at event time(s) : {2.0}
PDE computation time : 0.015 seconds

Swaption market price = 3.91644

------------------------------------------------------
Swaption 3y/12y = 4.85609 (closed-form price = 4.85507)
Rel. error : 0.0211%
PDE DF function called in payoff at event time(s) : {3.0}
PDE computation time : 0.019 seconds

Swaption market price = 4.85507

------------------------------------------------------
Swaption 4y/11y = 5.44950 (closed-form price = 5.44928)
Rel. e

## Pricing d'un swap callable 15y no call 2y

L'idée est d'utiliser le modèle pour évaluer un swap payeur 15y callable, avec une no call period de 2 ans et de mesurer le "pick-up" sur le taux fixe ainsi obtenu par rapport à un swap 15 ans qui ne serait pas callable.

Les données de marché utilisées sont celles figurant dans ce [pricer Excel](https://docs.google.com/spreadsheets/d/1e0tB87TtsNRobLgHoQPoTMN1nsKaEoxx/export?format=xlsm). Dans ce pricer, on peut évaluer les swaps ainsi que les swaptions européennes, si bien qu'il est possible de structurer un swap 15 ans *one time callable* dans 2 ans.

Sans possibilité d'annulation, le swap paye un taux fixe de 0.867% (taux swap de marché 15 ans figurant dans la courbe des taux).

En rendant ce swap annulable dans deux ans et structurant l'ensemble afin que la structure soit de valeur nulle aujourd'hui, le taux fixe de la structure passe à 1.365%, soit un pick-up de près de 50 points de base.

Ci-dessous, il s'agit sur les mêmes market datas, de structurer le swap multi-callable (possibilité d'annulation dans 2 ans, 3 ans, ..., 14 ans) : avec une mean reversion de 3%, le taux fixe de la structure passe à 1.63% soit un pick-up de 76 points de base par rapport au taux swap 15 ans.

In [ ]:
"""
Pricing d'un swap payeur 15y callable no call 2y (avec les market datas du Pricer excel mars 2016)

"""
import numpy as np
import math
import time
# from hw1f import hw1f_model, instrument
# from hw1f_pde import hw1f_pde
# from zc_curve import zc_curve

t0 = time.time()

# yield curve t = 0
maturities = np.array(
[0.0055,
0.2548,
0.5068,
1.0055,
1.5055,
3.0055,
4.0055,
5.0055,
6.0055,
7.0055,
8.0055,
9.0055,
10.0055,
11.0055,
12.0055,
13.0055,
14.0055,
15.0055,
16.0055,
17.0055,
18.0055,
19.0055,
20.0055,
21.0055,
22.0055,
23.0055,
24.0055,
25.0055,
26.0055,
27.0055,
28.0055,
29.0055,
30.0055])

zc_rates = np.array([
-0.3518,
-0.2487,
-0.1362,
-0.1422,
-0.1507,
-0.1165,
-0.0470,
0.0226,
0.1225,
0.2227,
0.3280,
0.4339,
0.5406,
0.6252,
0.7103,
0.7689,
0.8280,
0.8875,
0.9131,
0.9391,
0.9653,
0.9917,
1.0182,
1.0244,
1.0308,
1.0373,
1.0437,
1.0504,
1.0491,
1.0478,
1.0464,
1.0453,
1.0442]) * 0.01
yc = zc_curve(maturities, zc_rates)

# calibration sur set de swaptions diagonales 15y
total_mat = 15
strike = 0.0163 # choisi de telle sorte que PV total = PV swap  + PV swaption  = 0
# strike 1.375% si on se limite à la one time callable
calib_set = []
for expiry in range (2, total_mat) :
	tenor = total_mat - expiry
	swaption = instrument.swaption('receiver', expiry, tenor, strike, 100.)
	swaption.set_market_data(yc.df, 0.0069)
	calib_set.append(swaption)

# create hw1f model & calibrate
mean_rev = 0.03
hw = hw1f_model(mean_rev)
hw.calibrate_sigma(calib_set)
hw.print()
print("Calibration time = {:.3f} seconds\n\n".format(time.time() - t0))


# product event times => based on calib set as we price the associated bermuda
event_t = []
for swaption in calib_set :	event_t.append(swaption.exp_time)

# PDE params
nx = 601
nstdev = 5
dt = 0.05

# Price bermuda swaption based upon calibration set
def bermuda_payoff(t, v_t):
	for swaption in calib_set:
		if (math.isclose(t, swaption.exp_time)):
			v_t = np.maximum(pde.pv_underlying(swaption, t), v_t)
			break
	return v_t

print("\nBERMUDA SWAPTION BASED UPON CALIBRATION SET")
# get event times = swaptions expiry times
event_t = []
for swaption in calib_set :
	event_t.append(swaption.exp_time)

pde = hw1f_pde(yc.df, hw, nx, nstdev, dt, event_t)
bermuda_price = pde.compute_price(bermuda_payoff, 'Bermuda swaption 15y no call 2y')

# find most expensive swaption & compute switch option value...
swaptions_prices = np.empty(len(calib_set))
for i in range(len(calib_set)):
	swaptions_prices[i] = calib_set[i].market_price()
imax = np.where(swaptions_prices == np.amax(swaptions_prices))[0][0]
me = calib_set[imax] # most expensive swaption
print('Most expensive swaption : {}y/{}y, price = {:.6f}'.format(me.exp_time, int(me.pay_times[-1]-me.start_time), me.market_price()) )
print('Switch option = {:.6f}\n'.format(bermuda_price - me.market_price()))

# compute callable swap PV
payer_swap = instrument.swaption('payer', 0, total_mat, strike, 100.)
payer_swap.set_market_data(yc.df, 0.)
print("PV receiver bermuda swaption 15y no call 2y {:.4f}% = {:.4f}".format(100.*strike, bermuda_price))
print("PV payer swap 15y {:.4f}% = {:.4f}".format(100.*strike, payer_swap.pv_underlying()))
print("PV callable = {:.4f} (strike / payer swap fixed rate adjusted such that PV callable = 0)\n".format(bermuda_price + payer_swap.pv_underlying()))

print("Swap rate 15y = {:.4f}%".format(100.*payer_swap.forward()))
print("Callable fixed rate = {:.4f}%".format(strike*100))
print("Pick up = {:.2f}%\n\n".format(100*(strike-payer_swap.forward())))

------------------------------------
HW1F MODEL | Mean reversion = 3.00%
Vol sigma(t) (%) = 
[[ 0.      0.8602]
 [ 2.      0.8529]
 [ 3.      0.8484]
 [ 4.      0.8423]
 [ 5.      0.8404]
 [ 6.      0.8371]
 [ 7.      0.8359]
 [ 8.      0.8357]
 [ 9.      0.8383]
 [10.      0.8324]
 [11.      0.8363]
 [12.      0.8227]
 [13.      0.8193]
 [14.      0.8193]]

Calibration time = 0.190 seconds



BERMUDA SWAPTION BASED UPON CALIBRATION SET
------------------------------------------------------
Bermuda swaption 15y no call 2y = 10.98344
PDE DF function called in payoff at event time(s) : {2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0}
PDE computation time : 0.121 seconds

Most expensive swaption : 2y/13y, price = 9.423867
Switch option = 1.559572

PV receiver bermuda swaption 15y no call 2y 1.6300% = 10.9834
PV payer swap 15y 1.6300% = -10.9872
PV callable = -0.0038 (strike / payer swap fixed rate adjusted such that PV callable = 0)

Swap rate 15y = 0.8665%
Callable 